# Databases

This will be an effort to write to databases

You need sqlite.  Windows users probably need to download it.  It's very small and easy to configure.

Download here: https://www.sqlite.org/download.html



In [1]:
import sqlite3  # we use sqllite because most systems have it already installed.

In [2]:
conn = sqlite3.connect('example.db')

In [3]:
c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS stocks''')
# Create table
c.execute('''CREATE TABLE stocks
             (id integer primary key, date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES (1, '2006-01-05','BUY','RHAT',100,35.14)")


In [4]:
# TODO:  Insert some more rows into stocks table

c.execute("INSERT INTO stocks VALUES (2, '2019-01-05','BUY','AAPL',100,35.14)")
c.execute("INSERT INTO stocks VALUES (3, '2019-02-05','BUY','DELL',100,35.14)")


In [5]:

# Save (commit) the changes
conn.commit()


### Let's do some bulk inserts

We'll do some bulk inserts

In [6]:

t = ('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(c.fetchone())

# Larger example that inserts many records at a time
purchases = [(10,'2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             (11,'2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             (12, '2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?,?)', purchases)


(1, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [7]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)

(1, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
(2, '2019-01-05', 'BUY', 'AAPL', 100.0, 35.14)
(3, '2019-02-05', 'BUY', 'DELL', 100.0, 35.14)
(10, '2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
(12, '2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
(11, '2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


### Let's use pandas to read into a dataframe

This should be easier.  Make a dataframe by saying pd.read_sql.query

TODO: You can read the stocks table by saying "select * from stocks"

In [8]:
import pandas as pd
df = pd.read_sql_query("select * from stocks;", conn)
df

,id,date,trans,symbol,qty,price
0,1,2006-01-05,BUY,RHAT,100.0,35.14
1,2,2019-01-05,BUY,AAPL,100.0,35.14
2,3,2019-02-05,BUY,DELL,100.0,35.14
3,10,2006-03-28,BUY,IBM,1000.0,45.00
4,11,2006-04-05,BUY,MSFT,1000.0,72.00
5,12,2006-04-06,SELL,IBM,500.0,53.00


In [9]:
df = df.append(pd.DataFrame({'id': [21,22,23],
                        'date': ['2018-01-01', '2018-04-13', '2018-04-15'],
                        'trans': ['BUY', 'BUY', 'BUY'],
                        'symbol': ['AAPL', 'DELL', 'CSCO'],
                        'qty': [100.0, 200.0, 300.0],
                        'price': [45.0, 55.0, 66.0]
                       }))
df

,id,date,trans,symbol,qty,price
0,1,2006-01-05,BUY,RHAT,100.0,35.14
1,2,2019-01-05,BUY,AAPL,100.0,35.14
2,3,2019-02-05,BUY,DELL,100.0,35.14
3,10,2006-03-28,BUY,IBM,1000.0,45.00
4,11,2006-04-05,BUY,MSFT,1000.0,72.00
5,12,2006-04-06,SELL,IBM,500.0,53.00
0,21,2018-01-01,BUY,AAPL,100.0,45.00
1,22,2018-04-13,BUY,DELL,200.0,55.00
2,23,2018-04-15,BUY,CSCO,300.0,66.00


In [12]:
df.to_sql("stocks", conn, if_exists="replace", index=False) # write to table

### Do a read from the stocks table

Use pandas to read from the stocks table.

In [11]:
# TODO: DO a read from the stocks table and make sure the new rows are there?

df = pd.read_sql_query("select * from stocks;", conn, )
 # TODO:How to read from table.
df

,id,date,trans,symbol,qty,price
0,1,2006-01-05,BUY,RHAT,100.0,35.14
1,2,2019-01-05,BUY,AAPL,100.0,35.14
2,3,2019-02-05,BUY,DELL,100.0,35.14
3,10,2006-03-28,BUY,IBM,1000.0,45.00
4,11,2006-04-05,BUY,MSFT,1000.0,72.00
5,12,2006-04-06,SELL,IBM,500.0,53.00
6,21,2018-01-01,BUY,AAPL,100.0,45.00
7,22,2018-04-13,BUY,DELL,200.0,55.00
8,23,2018-04-15,BUY,CSCO,300.0,66.00


### ORM in Python

Python supports ORMs (Object-Relational Mapping) like other languages. Examples of ORMs are (N)Hibernate (Java/.NET), Spring Framework (Java), Entity Framework (.NET) and many others.

In Python, there are a couple of ORMs that are popular. One is the very popular django framework, which is an all-inclusive MVC framework similar to Spring in Java, both of which which also have a ORM (among a huge number of other features).

In keeping with the philosophy of "do one thing and do it well" SQLAlchemy is designed to provide a lightweight database layer + ORM for python.  We will look at SQLAlchemy.

In [13]:
from sqlalchemy import Column, ForeignKey, Float, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, create_session
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base


 
Base = automap_base()

engine = create_engine('sqlite:///example.db')




In [16]:
from sqlalchemy import *
meta = MetaData()
Stocks = Table('stocks', meta, autoload=True, autoload_with=engine)


### Query in SQLAlchemy ORM

Let's do a query of stocks in sqlalchemy ORM

In [18]:
session = create_session(bind=engine)

stocks = session.query(Stocks).all()    

for s in stocks:
    print((s.id,s.date,s.symbol,s.trans,s.price,s.qty))

(1, '2006-01-05', 'RHAT', 'BUY', 35.14, 100.0)
(2, '2019-01-05', 'AAPL', 'BUY', 35.14, 100.0)
(3, '2019-02-05', 'DELL', 'BUY', 35.14, 100.0)
(10, '2006-03-28', 'IBM', 'BUY', 45.0, 1000.0)
(11, '2006-04-05', 'MSFT', 'BUY', 72.0, 1000.0)
(12, '2006-04-06', 'IBM', 'SELL', 53.0, 500.0)
(21, '2018-01-01', 'AAPL', 'BUY', 45.0, 100.0)
(22, '2018-04-13', 'DELL', 'BUY', 55.0, 200.0)
(23, '2018-04-15', 'CSCO', 'BUY', 66.0, 300.0)


### Close the connection

Run this cell to close the connection

In [19]:
conn.close()